## Import packages

In [1]:
import pyomo
import pyomo.opt
import pyomo.environ as pyo
import numpy as np
import pandas as pd
import matplotlib as plt

In [2]:
n_time = 24

In [3]:
def _auxDictionary(a):
    temp_dictionary = {}
    if len(a.shape) == 3:
        for dim0 in np.arange(a.shape[0]):
            for dim1 in np.arange(a.shape[1]):
                for dim2 in np.arange(a.shape[2]):
                    temp_dictionary[(dim0+1, dim1+1, dim2+1)] = a[dim0, dim1, dim2]
    elif len(a.shape) == 2:
        for dim0 in np.arange(a.shape[0]):
            for dim1 in np.arange(a.shape[1]):
                temp_dictionary[(dim0+1, dim1+1)] = a[dim0, dim1]
    else:
        for dim0 in np.arange(a.shape[0]):
            temp_dictionary[(dim0+1)] = a[dim0]
    return temp_dictionary
#temp_dict1 = _auxDictionary(loadLimit)

## Data

In [4]:
data = {}
data['Inputs'] = pd.read_csv('Inputs.csv')
data['EVs_Inputs'] = pd.read_csv('EVs_Inputs.csv')
data['alpha'] = pd.read_csv('alpha.csv')
data['PchmaxEV'] = pd.read_csv('PchmaxEV.csv')
data['S'] = pd.read_csv('S.csv')

In [5]:
n_evs = data['EVs_Inputs']['Esoc'].size

## Start Time

In [6]:
from datetime import datetime

now = datetime.now()

start_time = now.strftime("%H:%M:%S")
print("Start Time =", start_time)

Start Time = 14:50:05


## Sets

In [7]:
model = pyo.ConcreteModel()

model.ev = pyo.Set(initialize = np.arange(1, n_evs + 1))
model.t = pyo.Set(initialize = np.arange(1, n_time + 1))


## Parameters

In [8]:
model.EEVmax = pyo.Param(model.ev, initialize =_auxDictionary(data['EVs_Inputs'].to_numpy()[:,2]))
model.EEVmin = pyo.Param(model.ev, initialize =_auxDictionary(data['EVs_Inputs'].to_numpy()[:,1]))
model.ESoc = pyo.Param(model.ev, initialize =_auxDictionary(data['EVs_Inputs'].to_numpy()[:,0]))
model.dT = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,0]))
model.cDA = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,1]))
model.Pl = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,2]))
model.Php = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,3]))
model.PchmaxEV = pyo.Param(model.ev, model.t, initialize =_auxDictionary(data['PchmaxEV'].to_numpy()))
model.S = pyo.Param(model.ev, model.t, initialize = _auxDictionary(data['S'].to_numpy()))
model.alpha = pyo.Param(model.ev, model.t, initialize = _auxDictionary(data['alpha'].to_numpy()))
model.RealHour = pyo.Param(model.t, initialize=_auxDictionary(data['Inputs'].to_numpy()[:,2]))
model.penalty1 = 1000
model.penalty2 = 1000
model.penalty3 = 0.6
model.Etrip = pyo.Param(model.ev, initialize=_auxDictionary(data['EVs_Inputs'].to_numpy()[:,3]))
model.n = 1
model.m = 0.6
model.Ppeak = 80
model.factor = 1

## Variables

In [9]:
model.PEV = pyo.Var(model.ev, model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.EEV = pyo.Var(model.ev, model.t, domain = pyo.Reals, initialize = 0)
model.Etriprelax = pyo.Var(model.ev, model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.Eminsocrelax = pyo.Var(model.ev, model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.Etripn = pyo.Var(model.ev, model.t, domain = pyo.Reals, initialize = 0)
model.Ppeakred = pyo.Var(model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.Ppeakred2 = pyo.Var(domain = pyo.Reals, initialize = 0)

## Constraints

In [10]:
def _balance_etripn(m,ev,t): 
    return m.Etripn[ev,t] == m.Etrip[ev]*m.S[ev,t]/(sum([m.S[ev,k] for k in np.arange(1, n_time + 1)]))
model.balance_etripn = pyo.Constraint(model.ev, model.t, rule = _balance_etripn)

def _balance_energy_EVS3(m,ev,t): 
    if t == 24:
        return m.EEV[ev,t] + m.Etriprelax[ev,t] >= m.EEVmax[ev]*m.m
    return pyo.Constraint.Skip
model.balance_energy_EVS3 = pyo.Constraint(model.ev, model.t, rule = _balance_energy_EVS3)

def _balance_energy_EVS(m,ev,t): 
    if t == 1:
        return m.EEV[ev,t] - m.Etriprelax[ev,t] == m.ESoc[ev] + m.PEV[ev,t]*m.dT[t]*m.n - m.Etripn[ev,t]
    return pyo.Constraint.Skip
model.balance_energy_EVS = pyo.Constraint(model.ev, model.t, rule = _balance_energy_EVS)

def _balance_energy_EVS2(m,ev,t): 
    if t > 1:
        return m.EEV[ev,t] - m.Etriprelax[ev,t] == m.EEV[ev,t-1] + m.PEV[ev,t]*m.dT[t]*m.n - m.Etripn[ev,t]
    return pyo.Constraint.Skip
model.balance_energy_EVS2 = pyo.Constraint(model.ev, model.t, rule = _balance_energy_EVS2)

def _power_charging_limit1(m,ev,t): 
    return m.PEV[ev,t] >= 0
model.power_charging_limit1 = pyo.Constraint(model.ev, model.t, rule = _power_charging_limit1)

def _power_charging_limit2(m,ev,t): 
    return m.PEV[ev,t] <= m.alpha[ev,t]*m.PchmaxEV[ev,t]*m.factor
model.power_charging_limit2 = pyo.Constraint(model.ev, model.t, rule = _power_charging_limit2)

def _energy_limits_EVS_1(m,ev,t): 
    return m.EEVmin[ev] <= m.EEV[ev,t] + m.Eminsocrelax[ev,t]
    #return m.EEVmin[ev] <= m.EEV[ev,t]
model.energy_limits_EVS_1 = pyo.Constraint(model.ev, model.t, rule = _energy_limits_EVS_1)

def _energy_limits_EVS_2(m,ev,t): 
    return m.EEV[ev,t] <= m.EEVmax[ev]
model.energy_limits_EVS_2 = pyo.Constraint(model.ev, model.t, rule = _energy_limits_EVS_2)

def _power_peak_s(m,ev,t): 
    return m.Pl[t] + sum(m.PEV[e,t] for e in np.arange(1, n_evs + 1)) - m.Ppeakred[t] <= m.Ppeak    
model.power_peak_s = pyo.Constraint(model.ev, model.t, rule = _power_peak_s)

def _energy_EVs_trips(m,t): 
    return m.Ppeakred2 >= m.Pl[t] + sum(m.PEV[ev,t] for ev in np.arange(1, n_evs + 1))
model.energy_EVs_trips = pyo.Constraint(model.t, rule = _energy_EVs_trips)

## Objective function

In [11]:
def _FOag(m):
    return sum(m.Ppeakred[t]**2 for t in np.arange(1, n_time + 1)) + sum([m.Etriprelax[ev,t]*m.penalty1 + m.Eminsocrelax[ev,t]*m.penalty2 for ev in np.arange(1, n_evs + 1) for t in np.arange(1, n_time + 1)])

model.FOag = pyo.Objective(rule = _FOag, sense = pyo.minimize)

## Solve the model

In [12]:
from pyomo.opt import SolverFactory

model.write('res_V4_EC.lp',  io_options={'symbolic_solver_labels': True})

# Create a solver
#opt = pyo.SolverFactory('cbc', executable='C:/Program Files/Cbc-releases.2.10.8-x86_64-w64-mingw64/bin/cbc.exe')

opt = pyo.SolverFactory('cplex', executable='C:/Program Files/IBM/ILOG/CPLEX_Studio221/cplex/bin/x64_win64/cplex.exe')
opt.options['LogFile'] = 'res_V4_EC.log'

#opt = pyo.SolverFactory('ipopt', executable='C:/Program Files/Ipopt-3.11.1-win64-intel13.1/bin/ipopt.exe')
#opt.options['print_level'] = 12
#opt.options['output_file'] = "res_V5_EC.log"

results = opt.solve(model)#, tee=True)
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: tmptnz5r71n
  Lower bound: 2170259773.8348765
  Upper bound: 2170259773.8348765
  Number of objectives: 1
  Number of constraints: 146379
  Number of variables: 103946
  Number of nonzeros: 18270027
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: 3.14
  Termination condition: optimal
  Termination message: Barrier - Optimal\x3a Objective = 2.1702597738e+09
  Error rc: 0
  Time: 11.583147048950195
# ----------------------------------------------------------
#   Solution Information
# ---------------

## Objective Function Value

In [14]:
pyo.value(model.FOag)

2170259773.8335614

## End Time

In [13]:
now = datetime.now()

end_time = now.strftime("%H:%M:%S")
print("End Time =", end_time)
print("Dif: {}".format(datetime.strptime(end_time, "%H:%M:%S") - datetime.strptime(start_time, "%H:%M:%S")))

End Time = 14:54:02
Dif: 0:03:57


## Cost Value

In [15]:
def ext_pyomo_vals(vals):
    # make a pd.Series from each
    s = pd.Series(vals.extract_values(),
                  index=vals.extract_values().keys())
    # if the series is multi-indexed we need to unstack it...
    if type(s.index[0]) == tuple:    # it is multi-indexed
        s = s.unstack(level=1)
    else:
        # force transition from Series -> df
        s = pd.DataFrame(s)
    return s

In [16]:
PEV_df = ext_pyomo_vals(model.PEV)
dT_df = ext_pyomo_vals(model.dT)
cDA_df = ext_pyomo_vals(model.cDA)
EEV_df = ext_pyomo_vals(model.EEV)

charge_cost = sum([PEV_df[t][ev]*dT_df[0][t]*cDA_df[0][t]
                   for ev in np.arange(1, n_evs + 1) for t in np.arange(1, n_time + 1)])

print('Charge cost: {}'.format(charge_cost))

Charge cost: 0.002826836133324312


## Results

In [17]:
print("Total Charge: {}".format(np.sum(PEV_df.to_numpy())))

Total Charge: 7.775559319730117e-05


In [18]:
EEV_df

,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
1,3800.0,3770.240000,3770.240000,3770.240001,3770.240001,3770.240001,3770.240001,3770.240001,3770.240001,3770.240002,...,3740.480003,3710.720003,3710.720003,3710.720003,3710.720003,3710.720004,3710.720004,3710.720004,3710.720004,4705.360002
2,2360.0,2358.640000,2358.640000,2358.640001,2358.640001,2358.640001,2358.640001,2358.640001,2358.640001,2358.640002,...,2358.640003,2357.280003,2357.280003,2357.280003,2357.280003,2357.280004,2357.280004,2357.280004,2357.280004,2948.640002
3,2360.0,2354.135000,2354.135000,2354.135001,2354.135001,2354.135001,2354.135001,2354.135001,2354.135001,2354.135002,...,2348.270003,2348.270003,2348.270003,2348.270003,2348.270003,2348.270004,2348.270004,2348.270004,2348.270004,2944.135002
4,2360.0,2285.370000,2285.370000,2285.370001,2285.370001,2285.370001,2285.370001,2285.370001,2285.370001,2285.370002,...,2136.110003,2136.110003,2136.110003,2136.110003,2136.110003,2136.110004,2136.110004,2136.110004,2136.110004,2838.055002
5,2720.0,2641.480000,2641.480000,2641.480001,2641.480001,2641.480001,2641.480001,2641.480001,2641.480001,2641.480002,...,2641.480003,2562.960003,2562.960003,2562.960003,2562.960003,2562.960004,2562.960004,2562.960004,2562.960004,3321.480002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
862,2080.0,2071.670000,2071.670000,2071.670001,2071.670001,2071.670001,2071.670001,2071.670001,2071.670001,2071.670002,...,2063.340003,2063.340003,2063.340003,2063.340003,2063.340003,2063.340004,2063.340004,2063.340004,2063.340004,2591.670002
863,3800.0,3479.960000,3159.920000,3159.920001,3159.920001,3159.920001,3159.920001,3159.920001,3159.920001,3159.920002,...,2839.880003,2519.840003,2519.840003,2519.840003,2519.840003,2519.840004,2519.840004,2519.840004,2519.840004,4109.920002
864,3400.0,2755.876667,2111.753334,2111.753334,2111.753334,2111.753334,2111.753334,2111.753335,2111.753335,2111.753335,...,2111.753336,1467.630003,1467.630003,1467.630003,1467.630003,1467.630004,1467.630004,1467.630004,1467.630004,3283.815002
865,540.0,539.010000,539.010000,539.010001,539.010001,539.010001,539.010001,539.010001,539.010001,539.010002,...,538.020003,538.020003,538.020003,538.020003,538.020003,538.020003,538.020004,538.020004,538.020004,674.010002


In [19]:
PEV_df

,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
1,0.0,4.933805e-09,4.728556e-09,4.794450e-09,5.023853e-09,5.492581e-09,6.276583e-09,7.294970e-09,8.031940e-09,8.496513e-09,...,6.942427e-09,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,4.934863e-09,4.729670e-09,4.795798e-09,5.025732e-09,5.495207e-09,6.279530e-09,7.297621e-09,8.034385e-09,8.498881e-09,...,6.944318e-09,5.996892e-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,4.934906e-09,4.729714e-09,4.795845e-09,5.025790e-09,5.495283e-09,6.279616e-09,7.297698e-09,8.034453e-09,8.498945e-09,...,6.944375e-09,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,4.934562e-09,4.729283e-09,4.795326e-09,5.025055e-09,5.494275e-09,6.278696e-09,7.297299e-09,8.034393e-09,8.499059e-09,...,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,4.934425e-09,4.729228e-09,4.795297e-09,5.025100e-09,5.494370e-09,6.278545e-09,7.296608e-09,8.033345e-09,8.497807e-09,...,6.943357e-09,5.997712e-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
862,0.0,4.935436e-09,4.730361e-09,4.796565e-09,5.026697e-09,5.496393e-09,6.280543e-09,7.298057e-09,8.034452e-09,8.498746e-09,...,6.944623e-09,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
863,0.0,0.000000e+00,4.727745e-09,4.793511e-09,5.022497e-09,5.490586e-09,6.274529e-09,7.293845e-09,8.031493e-09,8.496431e-09,...,6.941813e-09,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
864,0.0,0.000000e+00,4.727825e-09,4.793455e-09,5.022118e-09,5.489861e-09,6.274161e-09,7.294600e-09,8.032954e-09,8.498231e-09,...,6.942366e-09,6.002472e-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
865,0.0,4.932399e-09,4.727929e-09,4.794423e-09,5.024706e-09,5.494387e-09,6.278043e-09,7.294654e-09,8.030242e-09,8.493823e-09,...,6.941040e-09,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
PEV_df.sum()

1     4.693859e-07
2     3.547834e-06
3     3.585159e-06
4     3.659662e-06
5     3.865519e-06
6     4.243328e-06
7     4.855899e-06
8     5.651829e-06
9     6.223213e-06
10    6.592338e-06
11    6.793138e-06
12    6.884425e-06
13    6.009665e-06
14    4.865993e-06
15    4.212789e-06
16    2.387852e-06
17    4.844807e-07
18    4.796379e-07
19    4.751505e-07
20    4.855596e-07
21    4.954052e-07
22    4.924450e-07
23    4.987345e-07
24    4.961520e-07
dtype: float64

In [21]:
Etriprelax_df = ext_pyomo_vals(model.Etriprelax)

In [22]:
Etriprelax_df.sum()

1          0.000016
2       2364.213099
3       2536.691479
4       1977.258726
5       1639.251291
6       1492.697691
7       1390.605201
8       1301.852762
9       1213.645315
10      1144.366765
11      1100.563034
12      1053.462418
13      1024.742526
14      1007.342907
15      1007.102953
16       975.193066
17       818.076347
18       405.527511
19       298.944858
20       203.658435
21       183.161475
22       147.665384
23         0.000156
24    712986.993334
dtype: float64

In [23]:
Eminsocrelax_df = ext_pyomo_vals(model.Eminsocrelax)

In [24]:
Eminsocrelax_df.sum()

1     0.000000
2     0.000008
3     0.000078
4     0.000078
5     0.000078
6     0.000078
7     0.000078
8     0.000078
9     0.000078
10    0.000078
11    0.000078
12    0.000078
13    0.000078
14    0.000078
15    0.000078
16    0.000078
17    0.000078
18    0.000078
19    0.000078
20    0.000078
21    0.000078
22    0.000078
23    0.000078
24    0.000078
dtype: float64

In [25]:
Etripn_df = ext_pyomo_vals(model.Etripn)

In [26]:
Etripn_df.sum()

1         0.000000
2     99055.709983
3     54584.136167
4      7271.898119
5      3558.580983
6      2679.886649
7      2234.135530
8      1321.875530
9      1059.115952
10      351.191667
11     1321.875530
12     6730.330000
13    17814.336429
14        0.000000
15    31568.023864
16    80159.464983
17    16998.327649
18      834.574935
19      522.776364
20     4216.203167
21     8266.467333
22      799.099167
23        0.000000
24        0.000000
dtype: float64

In [27]:
Ppeakred_df = ext_pyomo_vals(model.Ppeakred)

In [28]:
Ppeakred_df

,0
1,8598.880000
2,9349.600004
3,9747.800004
4,9616.250004
5,9184.900004
6,8413.450004
7,7378.400005
8,6366.400006
9,5794.100006
10,5483.900007


In [29]:
Ppeakred2_df = ext_pyomo_vals(model.Ppeakred2)

In [30]:
Ppeakred2_df

,0
NaN,9827.800004


## Save results in csv files

In [31]:
import os 
folder = 'results_PeakS_b_' + str(n_evs)

if not os.path.exists(folder):
    os.makedirs(folder)
    
EEV_df.to_csv(folder + '/EEV.csv')
PEV_df.to_csv(folder + '/PEV.csv')
PEV_df.sum().to_csv(folder + '/PEV_h.csv')

Etriprelax_df.to_csv(folder + '/Etriprelax.csv')
Etriprelax_df.sum().to_csv(folder + '/Etriprelax_h.csv')

Eminsocrelax_df.to_csv(folder + '/Eminsocrelax.csv')
Eminsocrelax_df.sum().to_csv(folder + '/Eminsocrelax_h.csv')

Etripn_df.to_csv(folder + '/Etripn.csv')
Etripn_df.sum().to_csv(folder + '/Etripn_h.csv')

Ppeakred_df.to_csv(folder + '/Ppeakread.csv')